In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
os.environ["NEO4J_URI"] = "YOUR_URI"
os.environ["NEO4J_USERNAME"] = "YOUR_USERNAME"
os.environ["NEO4J_PASSWORD"] = "YOUR_PASSWORD"


In [ ]:
import pandas as pd

df = pd.read_csv("drug_reaction.csv")
print(df.head())

           Drug             Reaction
0  Atorvastatin             Polyuria
1  Atorvastatin  Uterine perforation
2  Atorvastatin       Liver disorder
3  Atorvastatin                Death
4  Atorvastatin       Rhabdomyolysis


In [ ]:
from langchain.schema import Document

documents = []
for idx, row in df.iterrows():
    content = f"Drug: {row['Drug']}\nReaction: {row['Reaction']}"
    documents.append(Document(page_content=content))


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

# Leveraging GPT-4o Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create Graph Documents
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

# Add them to Neo4j
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)


<ipython-input-5-f422aaa1bf71>:5: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Neo4jVector

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model="text-embedding-ada-002"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Entities(BaseModel):
    """Drug or Reaction names."""
    names: List[str] = Field(..., description="Names of drugs or reactions mentioned")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert on drug reactions. Extract drugs and reactions mentioned."),
    ("human", "Extract from this input: {question}")
])

entity_chain = prompt | llm.with_structured_output(Entities)


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1660: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:3})
            YIELD node, score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result


In [ ]:
def retriever(question: str):
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
"""
    return final_data


In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from typing import Tuple

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be standalone.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    (RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(run_name="HasChatHistoryCheck"),
     RunnablePassthrough.assign(chat_history=lambda x: _format_chat_history(x["chat_history"]))
     | CONDENSE_QUESTION_PROMPT
     | llm
     | StrOutputParser()
    ),
    RunnableLambda(lambda x: x["question"])
)

# Final prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use clear medical explanations if needed, be concise.
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Final chain
chain = (
    RunnableParallel({
        "context": _search_query | retriever,
        "question": RunnablePassthrough(),
    })
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger('neo4j').setLevel(logging.ERROR)


In [ ]:
result = chain.invoke({"question": "What type of adverse effects Atorvastatin can have on patients?"})
print(result)

Atorvastatin can cause a variety of adverse effects in patients, including:

1. **Metabolic Effects**: Such as metabolic syndrome and hypomagnesaemia.
2. **Hematological Issues**: Including thrombocytopenia, anaemia, and leukaemia.
3. **Neurological Symptoms**: Such as confusional state, depression, dementia, epilepsy, dysarthria, and balance disorders.
4. **Musculoskeletal Problems**: Including immune-mediated myositis, muscle discomfort, and muscle disorders.
5. **Gastrointestinal Disorders**: Such as abdominal pain, dyspepsia, and gastrointestinal haemorrhage.
6. **Respiratory Issues**: Including dyspnoea, asthma, and pneumonia.
7. **Cardiovascular Effects**: Such as hypotension, syncope, and myocarditis.
8. **Dermatological Reactions**: Including rash, alopecia, and application site pruritus.
9. **General Symptoms**: Such as fatigue, malaise, chills, pyrexia, and cold sweat.
10. **Other Reactions**: Including taste disorder, reperfusion arrhythmia, and idiopathic pulmonary fibrosis

In [ ]:
result = chain.invoke({"question": "What type of adverse effects Atorvastatin can have on patients with covid-19?"})
print(result)

Atorvastatin may lead to several adverse effects in patients with COVID-19, including:

1. **Respiratory Issues**: It can cause dyspnoea (difficulty breathing) and pneumonia, which are critical concerns in COVID-19 patients.
2. **Neurological Effects**: Symptoms such as confusion, fatigue, and dizziness may exacerbate the cognitive and physical challenges faced by COVID-19 patients.
3. **Immune Response**: Atorvastatin is associated with immune-mediated conditions, which could complicate the immune response to COVID-19.
4. **Metabolic Effects**: Conditions like metabolic syndrome and hypomagnesaemia may worsen the overall health status of patients with COVID-19.

These effects can complicate the management of COVID-19 and may require careful monitoring and adjustment of treatment.


In [ ]:
result = chain.invoke({"question": "Could Pravastatin drug have adverse effects on kidney"})
print(result)

Yes, Pravastatin can have adverse effects on the kidneys. It is associated with conditions such as Acute Kidney Injury and Chronic Kidney Disease, as indicated in the structured data. These effects may arise from the drug's impact on renal function, potentially leading to deterioration in kidney health.
